# Logistic Regression with Hyperparameter Optimization (scikit-learn)

<a href="https://colab.research.google.com/github/VertaAI/modeldb/blob/master/client/workflows/examples-without-verta/notebooks/sklearn-census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import itertools
import time

import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

---

## Prepare Data

In [2]:
try:
    import wget
except ImportError:
    !pip install wget # you may need pip3
    import wget

In [3]:
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.download(train_data_url)
test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.download(test_data_url)

In [4]:
df_train = pd.read_csv("census-train.csv")
X_train = df_train.iloc[:,:-1].values
y_train = df_train.iloc[:, -1]

df_train.head()

,age,capital-gain,capital-loss,hours-per-week,workclass_local-gov,workclass_private,workclass_self-emp-inc,workclass_self-emp-not-inc,workclass_state-gov,workclass_without-pay,...,occupation_handlers-cleaners,occupation_machine-op-inspct,occupation_other-service,occupation_priv-house-serv,occupation_prof-specialty,occupation_protective-serv,occupation_sales,occupation_tech-support,occupation_transport-moving,>50k
0,44,0,0,40,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,0,0,40,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,53,7298,0,60,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,49,0,0,40,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,53,0,1485,40,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


## Prepare Hyperparameters

In [5]:
hyperparam_candidates = {
    'C': [1e-4, 1e-1, 1, 10, 1e3],
    'solver': ['liblinear', 'lbfgs'],
    'max_iter': [15, 28],
}

# total models 20

# create hyperparam combinations
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

## Run Validation

In [6]:
# create validation split
(X_val_train, X_val_test,
 y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                             test_size=0.2,
                                                             shuffle=True)    

def run_experiment(hyperparams):
    
    # create and train model
    model = linear_model.LogisticRegression(**hyperparams)
    model.fit(X_train, y_train)
    
    # calculate and log validation accuracy
    val_acc = model.score(X_val_test, y_val_test)
    print(hyperparams, end=' ')
    print("Validation accuracy: {:.4f}".format(val_acc))
    
for hyperams in hyperparam_sets:
    run_experiment(hyperams)

{'C': 0.0001, 'solver': 'liblinear', 'max_iter': 15} Validation accuracy: 0.7941
{'C': 0.0001, 'solver': 'liblinear', 'max_iter': 28} Validation accuracy: 0.7999
{'C': 0.0001, 'solver': 'lbfgs', 'max_iter': 15} Validation accuracy: 0.7930
{'C': 0.0001, 'solver': 'lbfgs', 'max_iter': 28} Validation accuracy: 0.7919
{'C': 0.1, 'solver': 'liblinear', 'max_iter': 15} Validation accuracy: 0.7939
{'C': 0.1, 'solver': 'liblinear', 'max_iter': 28} Validation accuracy: 0.8459
{'C': 0.1, 'solver': 'lbfgs', 'max_iter': 15} Validation accuracy: 0.7928
{'C': 0.1, 'solver': 'lbfgs', 'max_iter': 28} Validation accuracy: 0.7930
{'C': 1, 'solver': 'liblinear', 'max_iter': 15} Validation accuracy: 0.7939
{'C': 1, 'solver': 'liblinear', 'max_iter': 28} Validation accuracy: 0.8470
{'C': 1, 'solver': 'lbfgs', 'max_iter': 15} Validation accuracy: 0.7928
{'C': 1, 'solver': 'lbfgs', 'max_iter': 28} Validation accuracy: 0.7930
{'C': 10, 'solver': 'liblinear', 'max_iter': 15} Validation accuracy: 0.7939
{'C': 1

## Pick the best hyperparameters and train the full data

In [7]:
best_hyperparams = {}
model = linear_model.LogisticRegression(multi_class='auto', **best_hyperparams)
model.fit(X_train, y_train)

LogisticRegression()

## Calculate Accuracy on Full Training Set

In [8]:
train_acc = model.score(X_train, y_train)
print("Training accuracy: {:.4f}".format(train_acc))

Training accuracy: 0.8432


---